# 测试训练好的模型的决策效果

In [1]:
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN,A2C
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
#check_env(env)

In [2]:
import numpy as np
from env import ElectricVehicleEnv
from stable_baselines3 import A2C
# 加载训练好的模型
#model = A2C.load("best_model/A2C/best_model.zip")
model = DQN.load("best_model/DQN/best_model.zip")
#model = A2C.load("model/a2c_electric_vehicle.zip")
#model = DQN.load("model/dqn_electric_vehicle.zip")
def print_state(state):
    """
    打印状态的每个属性，并标注其含义。
    """
    print(f"  当前节点 (Current Node): {state[0]}")
    print(f"  当前电池电量 (Battery): {state[1]*2}")
    print(f"  剩余时间 (Remaining Time): {state[2]/25}")
    print(f"  目标节点 (Target Node): {state[3]}")

def print_step_details(info):
    """
    打印每个步骤的详细信息。
    """
    print(f"  从节点 (From Node): {info['from_node']} 到节点 (To Node): {info['to_node']}")
    print(f"  距离缩短 (Distance_decrease): {info['distance_decrease']} km")
    print(f"  耗电 (Energy Consumed): {info['energy_consumed']} kWh")
    print(f"  是否充电 (Charging): {'Yes' if info['charging'] else 'No'}")
    if info['charging']:
        print(f"  充电量 (Energy Charged): {info['energy_charged']} kWh")
    print(f"  电量从 (Battery Start): {info['battery_start']} kWh 变成 (Battery End): {info['battery_end']} kWh")
    print(f"  剩余时间从 (Time Start): {info['time_start']} h 变成 (Time End): {info['time_end']} h")

# 测试模型

In [3]:

# 测试训练好的模型
obs, info = env.reset()
done = False

print("Initial State:")
print_state(obs)
step = 0
# 执行100步动作
for i in range(100):
    # 预测动作
    action, _states = model.predict(obs, deterministic=True)
    # 执行动作
    obs, reward, done,done1,  info = env.step(action)

    print("-" * 30)
    print(f"\nStep {step + 1}:")
    step+=1
    print_step_details(info)
    print_state(obs)
    print("Reward:", reward)
    print("Done:", done)
    
    if done:
        print("Episode finished! Resetting environment.")
        obs, info = env.reset(seed=14)
        print("Initial State:"+"-"*10)
        print_state(obs)
        step = 0


Initial State:
  当前节点 (Current Node): 28.0
  当前电池电量 (Battery): 47.0
  剩余时间 (Remaining Time): 1.934728546142578
  目标节点 (Target Node): 10.0
------------------------------

Step 1:
  从节点 (From Node): 28 到节点 (To Node): 21
  距离缩短 (Distance_decrease): -7.509704669999998 km
  耗电 (Energy Consumed): 1.08190857656 kWh
  是否充电 (Charging): Yes
  充电量 (Energy Charged): 16.099830008333335 kWh
  电量从 (Battery Start): 47.0 kWh 变成 (Battery End): 62.01792143177334 kWh
  剩余时间从 (Time Start): 1.934728501 h 变成 (Time End): 1.7737302009166667 h
  当前节点 (Current Node): 21.0
  当前电池电量 (Battery): 62.017921447753906
  剩余时间 (Remaining Time): 1.7737301635742186
  目标节点 (Target Node): 10.0
Reward: 26.268165233269826
Done: False
------------------------------

Step 2:
  从节点 (From Node): 21 到节点 (To Node): 10
  距离缩短 (Distance_decrease): 30.20384347 km
  耗电 (Energy Consumed): 4.2285380857999995 kWh
  是否充电 (Charging): No
  电量从 (Battery Start): 62.01792143177334 kWh 变成 (Battery End): 57.78938334597334 kWh
  剩余时间从 (Time Start): 